## Word2Vec as Matrix Factorization

In this assignment you are supposed to apply SVD to training your own [word embedding model](https://en.wikipedia.org/wiki/Word_embedding) which maps English words to vectors of real numbers.

Skip-Gram Negative Sampling (SGNS) word embedding model, commonly known as **word2vec** ([Mikolov et al., 2013](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)), is usually optimized by stochastic gradient descent. However, the optimization of SGNS objective can be viewed as implicit matrix factorization objective as was shown in ([Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf)).

###### 1. Notation
Assume we have a text corpus given as a sequence of words $\{w_1,w_2,\dots,w_n\}$ where $n$ may be larger than $10^{12}$ and $w_i \in \mathcal{V}$ belongs to a vocabulary of words $\mathcal{V}$. A word $c \in \mathcal{V}$ is called *a context* of word $w_i$ if they are found together in the text. More formally, given some measure $L$ of closeness between two words (typical choice is $L=2$), a word $c \in \mathcal{V}$ is called a context if $c \in \{w_{i-L}, \dots, w_{i-1}, w_{i+1}, \dots, w_{i+L} \}$ Let $\mathbf{w},\mathbf{c}\in\mathbb{R}^d$ be the *word embeddings* of word $w$ and context $c$, respectively. Assume they are specified by the mapping  $\Phi:\mathcal{V}\rightarrow\mathbb{R}^d$, so $\mathbf{w}=\Phi(w)$. The ultimate goal of SGNS word embedding model is to fit a good mapping $\Phi$.

Let $\mathcal{D}$ be a multiset of all word-contexts pairs observed in the corpus. In the SGNS model, the probability that word-context pair $(w,c)$ is observed in the corpus is modeled as the following distribution:

$$
P(\#(w,c)\neq 0|w,c) = \sigma(\mathbf{w}^\top \mathbf{c}) = \frac{1}{1 + \exp(-\mathbf{w}^\top \mathbf{c})},
$$

where $\#(w,c)$ is the number of times the pair $(w,c)$ appears in $\mathcal{D}$ and $\mathbf{w}^\top\mathbf{c}$ is the scalar product of vectors $\mathbf{w}$ and $\mathbf{c}$. Two important quantities which we will also use further are the number of times the word $w$ and the context $c$ appear in $\mathcal{D}$, which can be computed as

$$
\#(w) = \sum_{c\in\mathcal{V}} \#(w,c), \quad \#(c) = \sum_{w\in\mathcal{V}} \#(w,c).
$$


###### 2. Optimization objective

Vanilla word embedding models are trained by maximizing log-likelihood of observed word-context pairs, namely

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d}.
$$

Skip-Gram Negative Sampling approach modifies the objective by additionally minimizing the log-likelihood of random word-context pairs, so called *negative samples*. This idea incorporates some useful linguistic information that some number ($k$, usually $k=5$) of word-context pairs *are not* found together in the corpus which usually results in word embeddings of higher quality. The resulting optimization problem is

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + k \cdot \mathbb{E}_{c'\sim P_\mathcal{D}} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$

where $P_\mathcal{D}(c)=\frac{\#(c)}{|\mathcal{D}|}$ is a probability distribution over word contexts from which negative samples are drawn.

[Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf) showed that this objective can be equivalently written as

$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} f(w,c) = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$

A crucial observation is that this loss function depends only on the scalar product $\mathbf{w}^\top\mathbf{c}$ but not on embedding $\mathbf{w}$ and $\mathbf{c}$ separately.

###### 3. Matrix factorization problem statement

Let $|\mathcal{V}|=m$, $W \in \mathbb{R}^{m\times d}$ and $C \in \mathbb{R}^{m\times d}$ be matrices, where each row $\mathbf{w}\in\mathbb{R}^d$ of matrix $W$ is the word embedding of the corresponding word $w$ and each row $\mathbf{c}\in\mathbb{R}^d$ of matrix $C$ is the context embedding of the corresponding context $c$. SGNS embeds both words and their contexts into a low-dimensional space $\mathbb{R}^d$, resulting in the word and context matrices $W$ and $C$. The rows of matrix $W$ are typically used in NLP tasks (such as computing word similarities) while $C$ is ignored. It is nonetheless instructive to consider the product $W^\top C = M$. Viewed this way, SGNS can be described as factorizing an implicit matrix $M$ of dimensions $m \times m$ into two smaller matrices.

Which matrix is being factorized? A matrix entry $M_{wc}$ corresponds to the dot product $\mathbf{w}^\top\mathbf{c}$ . Thus, SGNS is factorizing a matrix in which each row corresponds to a word $w \in \mathcal{V}$ , each column corresponds to a context $c \in \mathcal{V}$, and each cell contains a quantity $f(w,c)$ reflecting the strength of association between that particular word-context pair. Such word-context association matrices are very common in the NLP and word-similarity literature. That said, the objective of SGNS does not explicitly state what this association metric is. What can we say about the association function $f(w,c)$? In other words, which matrix is SGNS factorizing? Below you will find the answers.

Solve SGNS optimization problem with respect to the $\mathbf{w}^\top\mathbf{c}$ and show that the matrix being factorized is

$$
M_{wc} = \mathbf{w}^\top\mathbf{c} = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} \right)
$$


**Note:** This matrix is called Shifted Pointwise Mutual Information (SPMI) matrix, as its elements can be written as

$$
\text{SPMI}(w,c) = M_{wc} = \mathbf{w}^\top\mathbf{c} = \text{PMI}(w,c) - \log k
$$

and $\text{PMI}(w,c) = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{\#(w)\cdot\#(c)} \right)$ is the well-known [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) of $(w,c)$.


$x=(w,c)$, $a=\#(w,c)$,$b=\#w$,$\gamma=\#c$

$\dfrac{df}{dx} = a\sigma(x)\exp(-x)+\frac{kb\gamma\sigma(-x)}{D}$

$y=\exp(-x)$
For quadratic equations we have two roots one equals to -1, other one to $$-\frac{kb\gamma}{Da}$$
So $$x=\log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} \right)$$

In [1]:
import os
import numpy as np
import subprocess
import sys
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [2]:
# Load enwik 8
data = np.loadtxt("data/enwik8.txt", dtype=str, delimiter='.')

# Create a list of sentences
sentences = [data[i].split() for i in data.nonzero()[0]]

2. Construct the word vocabulary from the obtained sentences which enumerates words which occur more than $r=200$ times in the corpus.

In [3]:
from collections import Counter
def create_vocabulary(sentences, r):
    voc=Counter()
    for i in range(len(sentences)):
        dic1={}
        wordfreq = [sentences[i].count(p) for p in sentences[i]]
        dic1=dict(zip(sentences[i],wordfreq))
        for k in dic1:
             voc[k]+=dic1[k]
            
    vocabluary=Counter()
    for p in voc:
        if voc[p]>=r:
            vocabluary[p]+=voc[p]
    
        
            
    return {word: (i, freq) for i, (word, freq) in enumerate(vocabluary.items())}

In [4]:
vocab =create_vocabulary(sentences,200)
vocabluary=dict(vocab)

3. Scan the text corpus with sliding window of size $5$ and step $1$ (which corresponds to $L$=2) and construct co-occurrence word-context matrix $D$ with elements $D_{wc}=\#(w,c)$. Ignoring words which occur less than $r=200$ times, but include them into the sliding window. Please, see the graphical illustration of the procedure described.

![Sliding window](sliding_window.png)

In [5]:
def create_corpus_matrix(sentences, vocab):
    vocab_size = len(vocab)
    cooccurrences = np.zeros((vocab_size, vocab_size),
                                      dtype=np.float64)
    L=2
    for line in sentences:
        words=[word for i, word in enumerate(line)]
        positions=[i for i, word in enumerate(line)]
        for k in positions:
            if words[k] in vocab:
                for right_i in range(k+1,min(len(words),k+1+L)):
                    if words[right_i] in vocab:
                        cooccurrences[vocab[words[k]][0],vocab[words[right_i]][0]]+=1 
                        #print("ir,jr=",vocab[words[k]][0],vocab[words[right_i]][0])
                                  
            
                for left_i in range(max(0,k-L),k):
                    if words[left_i] in vocab:
                        cooccurrences[vocab[words[k]][0],vocab[words[left_i]][0]]+=1
                        #print("il,jl=",vocab[words[k]][0],vocab[words[left_i]][0])
        words=[]  
        positions=[]
            
    return  cooccurrences

In [6]:
D = create_corpus_matrix(sentences,vocab)

In [7]:
import scipy as sp
from scipy.sparse import csr_matrix
T=csr_matrix(D)



4. To find good word embeddings, [Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf) proposed to find rank-$d$ SVD of Shifted Positive Pointwise Mutual Information (SPPMI) matrix

$$
U \Sigma V^\top \approx \text{SPPMI},
$$

where $\text{SPPMI}(w, c) = \max\left(\text{SPMI}(w, c), 0 \right)$ and $\text{SPMI}(w, c)$ is the element of the matrix $\text{SPPMI}$ at position $(w, c)$.


In [8]:
import scipy as sp
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds, eigs

def compute_embeddings(D, k, d):
    n_D=np.sum(D)
    n=D.shape[0]
    n_w=np.zeros((n,1))
    n_w=np.sum(D,axis=1).reshape(n,1)
    n_c=np.zeros((1,n))
    n_c=np.sum(D,axis=0).reshape(1,n)
    emb=np.maximum(np.log(n_D*D)-np.log(n_w.dot(n_c)*k),0)
    A=csr_matrix(emb)
    u,s,vt=svds(A, d)
    ksi=np.diag(s)
    embedding_matrix=u.dot(np.sqrt(ksi))
    
    
    return embedding_matrix

In [39]:
k = 5 # negative sampling parameter
W = compute_embeddings(D, k,200)

C:\Users\khurram\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


5. Write class **WordVectors** using provided template.

In [40]:

class WordVectors:
    
    def __init__(self, vocabulary, embedding_matrix):
        self.vocab = vocabulary
        self.W = embedding_matrix
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        
    def word_vector(self, word):
        """ 
        Takes word and returns its word vector.
        """
        i=self.vocab[word][0]
        word_vector=self.W[i]
        return word_vector
    
    def nearest_words(self, word, top_n=10):
        """ 
        Takes word from the vocabulary and returns its top_n
        nearest neighbors in terms of cosine similarity.
        """
        top=dict()
        w1=self.word_vector(word)
        for words in self.vocab.keys():
            w2=self.word_vector(words)
            top[words]=(np.dot(w1.T,w2))/(np.linalg.norm(w1,2)*np.linalg.norm(w2,2))
        
        sorted_by_value = sorted(top.items(),reverse=True, key=lambda kv: kv[1])
                            
                                         
        return sorted_by_value[1:11]

In [41]:
model = WordVectors(vocab, W)

In [49]:
model.word_vector("numerical").shape

(200,)

In [50]:
model.word_vector("rap").shape

(200,)

In [45]:
len(vocab.items())

5787

In [51]:
model.word_vector("rap").T.dot(model.word_vector("numerical"))

0.07692437137574666

In [42]:
model.nearest_words("numerical")

[('computation', 0.5465058022397276),
 ('mathematical', 0.5315278316790778),
 ('calculations', 0.49945706919609484),
 ('polynomial', 0.48538065620585513),
 ('calculation', 0.47300081538676186),
 ('practical', 0.46014260627224873),
 ('statistical', 0.4555170951220393),
 ('symbolic', 0.4549752576135998),
 ('geometric', 0.44109344247751925),
 ('simplest', 0.4379533370040826)]

In [32]:
model.nearest_words("linear")

[('differential', 0.7594258959609007),
 ('equations', 0.7239119589985703),
 ('equation', 0.6819562007574621),
 ('continuous', 0.6742284479542099),
 ('multiplication', 0.6736136605499364),
 ('integral', 0.6720418396042824),
 ('algebraic', 0.6671381718198981),
 ('vector', 0.6540550898958007),
 ('algebra', 0.6302501789190702),
 ('inverse', 0.6221151413953532)]

In [38]:
model.nearest_words("algebra")

[('calculus', 0.4458928642823162),
 ('geometry', 0.4384216472756925),
 ('equations', 0.3445289877801375),
 ('equation', 0.31410073600328614),
 ('topology', 0.3000525510451743),
 ('arithmetic', 0.2904635768230834),
 ('mathematics', 0.2861685612477852),
 ('differential', 0.2856547573415083),
 ('algebraic', 0.2806914372666656),
 ('integral', 0.2596334028163852)]

In [498]:
model.nearest_words("anarchism")

[('communism', 0.7916811289121348),
 ('anarcho', 0.7866274463952377),
 ('capitalism', 0.7835661470696771),
 ('socialism', 0.7521567332112503),
 ('liberalism', 0.7270942337271296),
 ('criticisms', 0.7045277321940145),
 ('capitalist', 0.6621936235916519),
 ('fascism', 0.5649384550241625),
 ('anarchist', 0.5273477653209915),
 ('marxist', 0.5183360211065385)]

In [496]:
model.nearest_words("ussr")

[('ukraine', 0.6713467751746154),
 ('russia', 0.6293371632068598),
 ('poland', 0.5504100164313704),
 ('belarus', 0.5384153768354997),
 ('yugoslavia', 0.538206195562453),
 ('romania', 0.517163403912315),
 ('serbia', 0.5066973712518864),
 ('austria', 0.49986558762309935),
 ('hungary', 0.46617419642332525),
 ('bulgaria', 0.42983248370363153)]

In [497]:
model.nearest_words("rap")

[('hop', 0.8283432918295232),
 ('hip', 0.8165784239590589),
 ('funk', 0.7577628668280326),
 ('rock', 0.7365094164302762),
 ('punk', 0.7063020806986656),
 ('music', 0.6759486602583847),
 ('pop', 0.6662904298474825),
 ('scene', 0.6589626455111179),
 ('band', 0.656824445005816),
 ('jazz', 0.6249604429360376)]